In [ ]:
%pip install --quiet git+https://github.com/ramayer/elephant-rumble-inference@v0.9.0-rc1

In [ ]:
import torch
import elephant_rumble_inference as eri
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'
aves_hubert_model          = eri.AvesTorchaudioWrapper().to(DEVICE)
elephant_rumble_classifier = eri.ElephantRumbleClassifier().to('cpu')
audio_file_processor       = eri.AudioFileProcessor(aves_hubert_model,elephant_rumble_classifier,device=DEVICE)
elephant_rumble_classifier.load_pretrained_weights('best.pth')


In [ ]:
audio_path = '/tmp/test.wav'
scores = audio_file_processor.classify_wave_file_for_rumbles(audio_path, limit_audio_hours=1)
feature_rate = audio_file_processor.rumble_sr // audio_file_processor.audio_samples_per_embedding

In [4]:
# Re-import in case module was modified
import importlib
import audio_classifier_visualizer as acv
importlib.reload(acv)
raven_file_helper = acv.RavenFileHelper()
audio_file_processor = acv.AudioFileProcessor()


In [ ]:
scores.shape

In [ ]:
import torch
from audio_classifier_visualizer import RavenLabel

audio_file = 'CEB1_20111010_000000.wav'
start_time = 0
duration = 60 * 60
AVES_SR = 4000
audio_path = '/tmp/test.wav'

sr = AVES_SR
# y = raven_file_helper.get_downsampled_tensor(audio_file,start_time,duration+start_time,new_sr=AVES_SR)

# Create some demo labels
demo_labels = [
    RavenLabel(
        bt=20.0,  # begin time
        et=30.0,  # end time 
        lf=200,   # low frequency
        hf=600,   # high frequency
        duration=10.0,
        audio_file=audio_path,
        t1="demo1",
        t2=None,
        t3=None,
        notes="Demo annotation 1",
        score=0.95,
        ravenfile=None
    ),
    RavenLabel(
        bt=100.0,
        et=120.0,
        lf=300,
        hf=900, 
        duration=20.0,
        audio_file=audio_path,
        t1="demo2",
        t2=None,
        t3=None,
        notes="Demo annotation 2", 
        score=0.85,
        ravenfile=None
    )
]
audio_file_visualizer = acv.AudioFileVisualizer(audio_path, start_time=0, end_time=60*5)


#y = raven_file_helper.load_entire_wav_file(audio_path, new_sr = AVES_SR)
#print(y.shape)
#print(sr)
#y = y.flatten().to(torch.float16)[0:10000]
audio_file_visualizer.visualize_audio_file_fragment(
    f"{audio_file} and scores",
    '/tmp/1.png',
    # audio_path,
    scores[0:468,1],
    scores[0:468,0],
    audio_file_processor,
    start_time=0,
    end_time=60*5,
    colormap='raw',
    width=16,
    height=3.5,
    labels=demo_labels  # Add the demo labels
)
from PIL import Image
import IPython.display as ipd
ipd.display(Image.open('/tmp/1.png'))
